## **Learning Objectives**

By completing these exercises, you will:

- Understand Retrieval-Augmented Generation (RAG) and its components.
- Load, preprocess, and handle PDF documents effectively.
- Convert textual data into embeddings for efficient retrieval.
- Implement and test document retrieval systems using LangChain and FAISS.
- Integrate retrieval systems with free Language Models (LLMs) from ChatGroq .
- Build an interactive chat-based Q&A system.

---

## **Exercise 1: Setup and Warm-up**

In this exercise, you'll set up your environment and select a suitable language model.

**Steps:**

1. **Load Environment Variables:** Ensure your environment variables (e.g., API keys, tokens) are securely stored and loaded.
2. **Choose LLM:** Select a free LLM model from from ChatGroq. 
3. **Instantiate the Model:** Create an instance of your chosen model.


In [10]:
# Import necessary libraries
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint

# Load environment variables
load_dotenv()



True

---

## **Exercise 2: Data Ingestion**

In this exercise, you'll learn to load PDF data into a Python environment.

**Steps:**

1. **Import PDF Loader:** Use LangChain’s `PyPDFLoader`.
2. **Load PDF File:** Create a function to read the PDF file.
3. **Display PDF Content:** Print the number of pages and first page content.

In [11]:
# Import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader

# Example function to load PDF

def load_pdf(pdf_path):
    pass  # Implement PDF loading here

In [16]:
# Load your PDF and print out content here

---

## **Exercise 3: Document Chunking**

This exercise introduces splitting large documents into manageable text chunks.

**Steps:**

1. **Import Text Splitter:** Use `RecursiveCharacterTextSplitter`.
2. **Chunk Document:** Write a function that splits loaded documents into chunks.
3. **Test Function:** Verify by displaying the resulting chunks.


In [17]:
# Import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Example chunking function
def chunk_documents(documents, chunk_size=200, chunk_overlap=50):
    pass  # Implement your chunking logic

In [18]:
# Execute your chunking function and display results here


---

## **Exercise 4: Embedding and Storage**

In this exercise, you will create embeddings from text chunks and store them efficiently.

**Steps:**

1. **Choose Embedding Model:** Use `sentence-transformers/all-mpnet-base-v2` from Hugging Face.
2. **Generate Embeddings:** Transform document chunks into embeddings.
3. **Store Embeddings:** Save these embeddings using FAISS locally.


In [19]:
# Import libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Example function for embeddings and storage
def embed_and_store(chunks):
    pass  # Implement your embedding creation and storage logic

In [20]:
# Generate embeddings and save them locally

---

## **Exercise 5: Retrieval from FAISS**

Here, you will learn how to retrieve documents from a vector database using embeddings.

**Steps:**

1. **Load Embeddings:** Load stored embeddings from the FAISS database.
2. **Implement Retrieval:** Create logic to retrieve relevant chunks based on queries.
3. **Test Retriever:** Execute retrieval using sample queries.

In [21]:
# Implement retrieval logic from your FAISS database

In [22]:
# Test your retrieval system with queries

---

## **Exercise 6: Connecting Retrieval with LLM**

You'll now connect document retrieval with the Language Model.

**Steps:**

1. **Create Retrieval Chain:** Link your retrieval system to your instantiated LLM.
2. **Test the Chain:** Confirm it works by generating answers from retrieved documents.

In [23]:
# Write a function to create retrieval and document processing chains


In [24]:
# Invoke your chain with a sample question

---

## **Exercise 7: Interactive Chat System**

In the final exercise, build an interactive chat-based query system.

**Steps:**

1. **Create Chat Interface:** Develop a simple function for interactive querying.
2. **Run the Chat:** Allow users to ask questions and receive immediate responses.


In [25]:
# Define your interactive chat querying function

In [26]:
# Run and test your interactive chat system

---

## **Conclusion & Reflection**

After completing these exercises:

- Summarize key concepts learned.
- Reflect on the effectiveness and limitations of the free LLM and RAG system you've built.
- Consider how you might improve or extend your system in practical applications.

---